In [ ]:
import pandas as pd
import numpy as np
from glob import glob
# from tqdm import tqdm
from tqdm import notebook
import mysql.connector
# from sqlalchemy import create_engine

import re
import json

In [ ]:
# DB설정
with open('./path.json') as file:
    json_obj = json.load(file)
    
con = mysql.connector.connect(host = json_obj['DB']['Host'],
                              port = json_obj['DB']['Port'],
                              database = 'capstone',
                              user = json_obj['DB']['User'],
                              password = str(json_obj['DB']['Password'])
)

In [ ]:
cursor = con.cursor(dictionary=True)

In [ ]:
cursor.execute('show Databases;')
result = cursor.fetchall()
pd.DataFrame(result)

In [ ]:
apt = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
officetel = pd.read_csv("./data/실거래_가공/오피스텔_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
yeon = pd.read_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv',index_col = 0, encoding='cp949')

In [ ]:
print(len(apt))
print(len(officetel))
print(len(yeon))

print(set(apt.columns).difference(set(yeon.columns))) # 단지명
print(set(yeon.columns).difference(set(apt.columns))) # 건물명

In [ ]:
import re
import json

re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")


In [ ]:
def parse(address, how='jibun'):
    '''
    주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

    address : 주소
    how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
    '''
    parts = address.split()
    sido = ""
    sigungu = ""
    ilbangu = ""
    eupmyeondong = ""
    ri_roadname = ""
    for idx, part in enumerate(parts):
        if idx == 0:
            sido = part
            continue
        
        if part.endswith("시") or part.endswith("군"):
            sigungu = part
            continue
        
        if part.endswith("구"):
            if idx == 1:
                if len(part) >= 4 and re.search(re1, part):
                    sigungu = part[0:2] + "시"
                    ilbangu = part[2:]
                else:
                    sigungu = part
            else:
                ilbangu = part
            continue
        
        if re.search(re2,
                        part) and eupmyeondong == '' and how == 'jibun':
            eupmyeondong = part
            continue
            
        if part.endswith("리") and how == 'jibun':
            ri_roadname = part
            continue
        
        if (part.endswith('로') or part.endswith('길')
            ) and how == 'road' and ri_roadname == '':
            ri_roadname = part
            continue
        
        if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
            ri_roadname = part
            continue

    return sido, sigungu, ilbangu, eupmyeondong, ri_roadname


In [ ]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(apt['시군구'].iloc[i]) for i in notebook.tqdm(range(len(apt)))]
officetel[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(officetel['시군구'].iloc[i]) for i in notebook.tqdm(range(len(officetel)))]
yeon[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(yeon['시군구'].iloc[i]) for i in notebook.tqdm(range(len(yeon)))]

In [ ]:
apt['갱신요구권 사용'].value_counts()

In [ ]:
apt['계약기간'].value_counts()

In [ ]:
print(len(apt[apt['계약기간'] == "-"]))
print(len(apt[apt['계약기간'] != '-']))

In [ ]:
apt.replace({'계약기간' : {'-' : '0~0'}},inplace=True) # 공란 값 변경

In [ ]:
apt['계약기간'].value_counts()

In [ ]:
apt[['계약시작일','계약종료일']] = apt['계약기간'].str.split("~", expand = True)

In [ ]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]]

In [ ]:
apt.columns # ri_roadname 리

In [ ]:
apt['갱신요구권 사용'].value_counts()

In [ ]:
apt['도로명'].astype(str).str.len().value_counts()

In [ ]:
yeon[yeon['층'].astype(str).str.len() == 3]['층']

# yeon['층'].astype(int)

In [ ]:
apt['층'].astype(str).str.len().value_counts() # 읍면동 최대 6자리 시군구 최대 4자리 시도 5자리 

In [ ]:
apt['계약구분'].value_counts()

In [ ]:
apt['전용면적(㎡)'].astype(str).str.len().value_counts() 

In [ ]:
cursor.execute("""
               SELECT TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,COLUMN_COMMENT
               FROM INFORMATION_SCHEMA.COLUMNS
               WHERE TABLE_SCHEMA = 'capstone'
               AND TABLE_NAME   = 'rent_price';
               """)

In [ ]:
result = pd.DataFrame(cursor.fetchall())

In [ ]:
db_column = list(result.COLUMN_NAME.values)

In [ ]:
print(result.COLUMN_COMMENT.values)

In [ ]:
# 토지고유번호,sido,sigungu,ilbangu,eupmyeondong,ri,본번,부번,산여부,전월세구분,전용면적,계약일자,보증금,월세,종전계약 보증금,종전계약 월세, 건물명,도로명,층,건축년도, 계약 시작일, 계약 종료일, 계약 구분
insert_df = apt[["토지고유번호","sido","sigungu","ilbangu","eupmyeondong","ri_roadname","본번","부번",'산',"전월세구분","전용면적(㎡)","계약년월","계약일","보증금(만원)","월세(만원)","종전계약 보증금 (만원)",'종전계약 월세 (만원)',"단지명","도로명","층","건축년도","계약시작일",'계약종료일','계약구분','갱신요구권 사용']]

In [ ]:
insert_df['계약구분'].value_counts()

['address',
 'sido',
 'sigungu',
 'ilbangu',
 'eupmyeondong',
 'ri',
 'main_num',
 'sub_num',
 'is_mountain',
 'rent_type',
 'exclusive_area',
 'contract_date',
 'deposit',
 'rent',
 'previous_deposit',
 'previous_rent',
 'building_name',
 'road_name',
 'floor',
 'construction_year',
 'contract_start',
 'contract_end',
 'contract_type',
 'renewal_right']

 Index(['토지고유번호', 'sido', 'sigungu', 'ilbangu', 'eupmyeondong', 'ri_roadname',
       '본번', '부번', '산', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세(만원)',
       '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '단지명', '도로명', '층', '건축년도', '계약시작일',
       '계약종료일', '계약구분'],
      dtype='object')

In [ ]:
insert_df['계약일자'] = [str(insert_df['계약년월'].iloc[i]).zfill(2) + str(insert_df['계약일'].iloc[i]).zfill(2) for i in notebook.tqdm(range(len(insert_df)))]

In [ ]:
insert_df['계약일자'].value_counts()

In [ ]:
insert_df['산'].value_counts()

In [ ]:
db_column

In [ ]:
insert_df['본번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['본번']]
insert_df['부번'] = [str(int(round(float(i),0))).zfill(4) for i in insert_df['부번']]

In [ ]:
insert_df['층'].isna().value_counts()

In [ ]:
import pandas as pd


def mass_inserter(args):
    df,data_type = args[0],args[1] # 데이터 프레임,데이터 타입
    cursor = con.cursor(dictionary=True)
    '''
    매칭 완료한 데이터를 넣는 함수

    df : 매칭 완료한 데이터
    data_type : 데이터 종류 (예 : '아파트-전세', '아파트-매매')
    jdbc : JDBC 객체
    '''
    insert_query = """
        INSERT INTO rent_price(
        address,
        sido,
        sigungu,
        ilbangu,
        eupmyeondong,

        ri,
        main_num,
        sub_num,
        is_mountain,
        rent_type,
        
        exclusive_area,
        contract_date,
        deposit,
        rent,        
        previous_deposit,

        previous_rent,
        property_type,
        building_name,
        road_name,
        floor,
        
        construction_year,
        contract_start,
        contract_end,
        contract_type,
        renewal_right
        
        ) VALUES(
        %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s)
    """

    def row_inserter(row,data_type, data_len):
        '''
        각 row 별로 insert 작업 수행

        row : DataFrame의 row
        jdbc : JDBC 객체
        data_type : 데이터 종류 (예 : '아파트-전세', '아파트-매매')
        data_len : 총 데이터 수
        '''
    
        insert_tuple = (
            None if pd.isna(row['토지고유번호']) else row['토지고유번호'],  # address_id
            None if pd.isna(row['sido']) else row['sido'], # 시도
            None if pd.isna(row['sigungu']) else row['sigungu'], # 시군구
            None if pd.isna(row['ilbangu']) else row['ilbangu'], # 일반구
            None if pd.isna(row['eupmyeondong']) else row['eupmyeondong'], # 읍면동
            None if pd.isna(row['ri_roadname']) else row['ri_roadname'], # 리
            None if pd.isna(row['본번']) else str(row['본번']),  # main_num
            None if pd.isna(row['부번']) else str(row['부번']),  # sub_num
            row['산'], # 산여부(1:일반, 2:산)
            None if pd.isna(row['전월세구분']) else row['전월세구분'], # rent_type
            None if pd.isna(row['전용면적(㎡)']) else row['전용면적(㎡)'], # exclusive_area
            None if pd.isna(row['계약일자']) else str(row['계약일자']), # contract_date = 계약년월 + 계약일
            None if pd.isna(row['보증금(만원)']) else int(str(row['보증금(만원)']).replace(',', '')), # deposit 
            None if pd.isna(row['월세(만원)']) else int(str(row['월세(만원)']).replace(',', '')), # rent
            None if pd.isna(row['종전계약 보증금 (만원)']) else int(str(row['종전계약 보증금 (만원)']).replace(',','')), # previous_deposit
            None if pd.isna(row['종전계약 월세 (만원)']) else int(str(row['종전계약 월세 (만원)']).replace(',','')), # previous_rent
            '아파트', # property_type(주택 유형)
            None if pd.isna(row['단지명']) else row['단지명'],  # apart_name
            None if pd.isna(row['도로명']) else row['도로명'],
            None if pd.isna(row['층']) else int(float(row['층'])),  # floor
            None if pd.isna(row['건축년도']) else str(int(row['건축년도'])),
            None if pd.isna(row['계약시작일']) else int(row['계약시작일']),
            None if pd.isna(row['계약종료일']) else int(row['계약종료일']),
            None if row['계약구분'] == '-' else str(row['계약구분']), # 계약구분(신규,갱신,Null)
            None if row['갱신요구권 사용'] == '-' else row['갱신요구권 사용']
            )

        cursor.execute(insert_query, insert_tuple)
        try: # 하루의 변동사항이 10건 이하인 경우  ZeroDivisionError: integer division or modulo by zero 발생, 임시 우회
            index = row.name + 1
            if index % (data_len // 10) == 0:
                print(f'{data_type} : {index/data_len*100:.2f}% a_id INSERT 완료')
        except:
            pass

    print(f'{data_type} 데이터 넣기 시작')
    data_len = len(df)
    df.progress_apply(row_inserter,
             data_type=data_type,
             data_len=data_len,
             axis=1)
    con.commit()
    print('커밋 후 커서를 종료했습니다.')
    cursor.close()
    con.close()
    print(f'{data_type} 데이터 넣기 완료')


In [ ]:
# mass_inserter 안에 row_inserter를 넣으면 됌
# 이중 정의 함수의 문제 상황은 어떤 게 발생할 수 있나??

In [ ]:
sample = insert_df.sample(5)

In [ ]:
sample

In [ ]:
from tqdm import tqdm
from multiprocessing import Pool

tqdm.pandas()

In [ ]:
args = [[insert_df.iloc[:400000],'아파트-전세'],
        [insert_df.iloc[400000:800000],'아파트-전세'],
        [insert_df.iloc[800000:1200000],'아파트-전세'],
        [insert_df.iloc[1200000:1600000],'아파트-전세'],
        [insert_df.iloc[1600000:],'아파트-전세']]

In [ ]:
from module.rent_price import mass_inserter as m

In [ ]:
if __name__ == '__main__':
    with Pool(5) as pool:
        with notebook.tqdm(total = len(args)) as pbar:
            for _ in pool.imap_unordered(m.mass_inserter,args):
                pbar.update()

In [29]:
l = pd.read_csv('./data/실거래_가공/아파트_가공_직접_삽입.csv',encoding='cp949')

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_41995/2987946642.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  l = pd.read_csv('./data/실거래_가공/아파트_가공_직접_삽입.csv',encoding='cp949')


In [30]:
l.index

RangeIndex(start=0, stop=2024191, step=1)

In [35]:
l.to_csv('./data/실거래_가공/아파트_가공_직접_삽입_utf-8.csv',encoding='utf-8')

In [36]:
ll = pd.read_csv('./data/실거래_가공/아파트_가공_직접_삽입_utf-8.csv',encoding='utf-8')

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_41995/1636326145.py:1: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ll = pd.read_csv('./data/실거래_가공/아파트_가공_직접_삽입_utf-8.csv',encoding='utf-8')


In [37]:
ll.index

RangeIndex(start=0, stop=2024191, step=1)

In [ ]:
mass_inserter(args = [insert_df,'아파트-전세']) # 해당 방식 15it/s 너무 느림

In [ ]:
import re
import json


class AddressParser:
    '''
    주소를 각 명칭으로 나누거나
    토지고유번호를 매칭해주는 class
    '''
    __jibun_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND eupmyeondong = %s 
        AND ri = %s 
        AND is_mountain = %s 
        AND num1 = %s
        AND {}
    '''
    __road_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND road_name = %s
        AND road_num1 = %s
        AND road_num2 = %s
    '''
    __search_by_aid = '''
        SELECT {}
        FROM addresses
        WHERE id = %s
    '''

    def __init__(self):
        '''
        주소를 불러와
        sido, sigungu, ilbangu, eupmyeondong, ri_roadname으로 나누는 작업을 해주는 class
        '''
        self.re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
        self.re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")

    def parse(self, address, how='jibun'):
        '''
        주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

        address : 주소
        how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
        '''
        parts = address.split()
        sido = ""
        sigungu = ""
        ilbangu = ""
        eupmyeondong = ""
        ri_roadname = ""
        for idx, part in enumerate(parts):
            if idx == 0:
                sido = part
                continue
            
            if part.endswith("시") or part.endswith("군"):
                sigungu = part
                continue
            
            if part.endswith("구"):
                if idx == 1:
                    if len(part) >= 4 and re.search(self.re1, part):
                        sigungu = part[0:2] + "시"
                        ilbangu = part[2:]
                    else:
                        sigungu = part
                else:
                    ilbangu = part
                continue
            
            if re.search(self.re2,
                         part) and eupmyeondong == '' and how == 'jibun':
                eupmyeondong = part
                continue
                
            if part.endswith("리") and how == 'jibun':
                ri_roadname = part
                continue
            
            if (part.endswith('로') or part.endswith('길')
                ) and how == 'road' and ri_roadname == '':
                ri_roadname = part
                continue
            
            if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
                ri_roadname = part
                continue

        return sido, sigungu, ilbangu, eupmyeondong, ri_roadname

    def get_address_id(self,
                       addr,
                       http,
                       jdbc,
                       silent=False,
                       additional_columns=None):
        '''
        지번 또는 도로명 주소로 구성 되어있는 주소를 parsing하여 토지 고유번호 매칭.
        깔끔하게 구성한 주소여야 함.
        대량의 주소를 변경하는 경우로 예상하여 http나 jdbc 객체를 받게끔 정의

        addr : 주소
        http : urllib3 PoolManager 객체
        jdbc : JDBC 객체
        silent : 못 찾았을 때, 출력값 보여주기 여부
        additional_columns : addresses에서 id 값 뿐만 아니라 더 검색할 columns 값들
        '''
        search_columns = 'id, '
        if additional_columns is not None:
            for column in additional_columns:
                search_columns += column + ', '
        search_columns = search_columns[:-2]

        def search_road(addr, splitted_addr):
            '''
            주소 형태가 도로명 주소인 경우라고 판단 할 때, 매칭하는 코드
            먼저 postcodify에서 검색 후 없으면 addresses에서 검색

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            road_name = splitted_addr[-1] + ' ' + number

            number = number.split('-')
            num1 = number[0]
            num2 = number[1] if len(number) == 2 else 0

            r = http.request('GET',
                             'http://192.168.20.2/postcodify/lib/search.php',
                             fields={'q': addr})

            if r.status == 200:
                results = json.loads(r.data.decode('utf8'))['results']
                filtered = list(
                    filter(lambda x: x['ko_doro'] == road_name, results))

                if len(filtered) == 1:
                    a_id = filtered[0]['address_id']
                    res = jdbc.fetch_data(
                        AddressParser.__search_by_aid.format(search_columns),
                        (a_id, ),
                        ret_type='raw')
                    if len(res) == 1:
                        if additional_columns is None:
                            return res[0][0]
                        return res[0]

                res = jdbc.fetch_data(
                    AddressParser.__road_search_query.format(search_columns),
                    splitted_addr[:3] + (splitted_addr[-1], num1, num2),
                    ret_type='raw')
                if len(res) == 1:
                    if additional_columns is None:
                        return res[0][0]
                    return res[0]

                return None
            raise Exception('도로명 주소 서버 통신 실패')

        def search_jibun(addr, splitted_addr):
            '''
            주소 형태가 지번 주소인 경우라고 판단 할 때, 매칭하는 코드

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            if number[0] == '산':
                number = number[1:]
                is_mountain = 2
            else:
                is_mountain = 1
            number = number.split('-')
            num1 = number[0]
            num2_case = f'num2 = {number[1]}' if len(
                number) == 2 and number[1].isdigit() else '(num2 IS NULL or num2 = 0)'

            res = jdbc.fetch_data(AddressParser.__jibun_search_query.format(
                search_columns, num2_case),
                                  splitted_addr + (is_mountain, num1),
                                  ret_type='raw')
            if len(res) == 1:
                if additional_columns is None:
                    return res[0][0]
                return res[0]
            return None

        match = re.compile(r'\s+산{0,1}\d{1,}-?\d{0,}').search(addr)

        if match is None:
            return None

        used_addr = addr[:match.end()]

        road_splited_addr = self.parse(used_addr, how='road')
        jibun_splited_addr = self.parse(used_addr, how='jibun')
        addr_type = 'road'
        if road_splited_addr[-1] == '':
            addr_type = 'jibun'

        if addr_type == 'road':
            result = search_road(used_addr, road_splited_addr)
            if result is None:
                result = search_jibun(used_addr, jibun_splited_addr)
        else:
            result = search_jibun(used_addr, jibun_splited_addr)
            if result is None:
                result = search_road(used_addr, road_splited_addr)
        if result is None and not silent:
            print(f'"{used_addr}" 주소 하나로 결정 불가')
        return result
